# Problem 4: SARIMAX Model for Real GDP, Consumption and Investment

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/YOUR_REPO/blob/main/Quantlet_Problem04_SARIMAX.ipynb)

---

**QuantLet Name:** `SSM_SARIMAX_GDP`  
**Published in:** State Space Models and Markov Switching Models — Chapter 8  
**Author:** Daniel Traian Pele  
**Institution:** Bucharest University of Economic Studies  
**Date:** February 2026  

---

## Problem
Fit a SARIMAX model to the `realGdpConsInv` dataset from the Federal Reserve Bank of St. Louis. Use `realcons` and `realinv` as exogenous regressors for `realgdp`. Conduct residual diagnostics.

---
## Plotting: ✅ Transparent background · ✅ No grid · ✅ Legend outside bottom


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'axes.grid': False,
    'font.size': 11, 'axes.labelsize': 12,
    'axes.titlesize': 13, 'figure.figsize': (12, 5)
})
print('Setup complete.')


In [ ]:
def plot_ljungbox_pvalues(series, noestimatedcoef=0, nolags=25,
                          title='P-values for Ljung-Box Test', figsize=(12, 5)):
    lags = np.arange(1, nolags + 1)
    lb_results = acorr_ljungbox(series, lags=nolags, model_df=noestimatedcoef)
    pvalues = lb_results['lb_pvalue'].values
    fig, ax = plt.subplots(figsize=figsize)
    fig.patch.set_alpha(0); ax.patch.set_alpha(0); ax.grid(False)
    ax.scatter(lags, pvalues, color='steelblue', s=40, zorder=3)
    ax.axhline(y=0.05, color='red', linestyle='--', linewidth=1.0, label='5% significance')
    ax.set_xlabel('Lag'); ax.set_ylabel('P-value')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_ylim(-0.05, 1.05)
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13), ncol=1, frameon=False)
    plt.tight_layout(rect=[0, 0.08, 1, 1])
    return fig, ax

print('Helper function defined.')


## 1. Load and Transform Data


In [ ]:
import statsmodels.formula.api as smf
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the dataset
# If PythonTsa is installed:
try:
    from PythonTsa.datadir import getdtapath
    dtapath = getdtapath()
    GCI = pd.read_csv(dtapath + 'realGdpConsInv.csv')
except:
    # Alternative: load from statsmodels or URL
    dta = sm.datasets.macrodata.load_pandas().data
    GCI = dta[['realgdp', 'realcons', 'realinv']].copy()

dates = pd.date_range('1959-03-31', periods=len(GCI), freq='Q')
GCI.index = dates

# Drop unnamed column if present
if 'Unnamed: 0' in GCI.columns:
    GCI = GCI.drop(columns=['Unnamed: 0'])

# Log transformation
LGCI = np.log(GCI)
LGCI.columns = ['Lrealgdp', 'Lrealcons', 'Lrealinv']

print(LGCI.describe())


In [ ]:
# Time series plot of logged variables
fig, ax = plt.subplots(figsize=(14, 6))
fig.patch.set_alpha(0); ax.patch.set_alpha(0); ax.grid(False)

ax.plot(LGCI['Lrealgdp'], color='steelblue', linewidth=1.2, label='Log Real GDP')
ax.plot(LGCI['Lrealcons'], color='darkorange', linewidth=1.2, label='Log Real Consumption')
ax.plot(LGCI['Lrealinv'], color='seagreen', linewidth=1.2, label='Log Real Investment')
ax.set_title('Logged Real GDP, Consumption and Investment', fontsize=13, fontweight='bold')
ax.set_ylabel('Log value')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), ncol=3, frameon=False)
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.show()


## 2. Preliminary OLS Regression


In [ ]:
# OLS regression of Lrealgdp on Lrealcons and Lrealinv
Llinearmod = smf.ols('Lrealgdp ~ Lrealcons + Lrealinv', data=LGCI).fit()
print(Llinearmod.summary())


In [ ]:
# KPSS stationarity test on residuals
kpss_result = sm.tsa.kpss(Llinearmod.resid, regression='c', nlags='auto')
print(f'KPSS statistic: {kpss_result[0]:.4f}')
print(f'P-value: {kpss_result[1]:.4f}')
print(f'Critical values: {kpss_result[3]}')
print(f'\nResiduals are stationary? {"YES ✅" if kpss_result[1] >= 0.05 else "NO ❌"}')


In [ ]:
# ACF and PACF of OLS residuals
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.patch.set_alpha(0)
for ax in axes:
    ax.patch.set_alpha(0); ax.grid(False)

plot_acf(Llinearmod.resid, lags=36, ax=axes[0], color='steelblue',
         vlines_kwargs={'colors': 'steelblue'})
axes[0].set_title('ACF of OLS Residuals', fontsize=12, fontweight='bold')

plot_pacf(Llinearmod.resid, lags=36, ax=axes[1], color='darkorange',
          vlines_kwargs={'colors': 'darkorange'})
axes[1].set_title('PACF of OLS Residuals', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()
print('PACF cuts off after lag 1 → use SARIMAX(1,0,0)(0,0,0)_4')


## 3. SARIMAX(1,0,0)(0,0,0)$_4$ Model


In [ ]:
# Fit SARIMAX model
X = LGCI[['Lrealcons', 'Lrealinv']]
Y = LGCI['Lrealgdp']
X = sm.add_constant(X, prepend=False)

sarimaxmod = SARIMAX(endog=Y, exog=X, order=(1, 0, 0),
                     seasonal_order=(0, 0, 0, 4)).fit(method='bfgs', disp=False)
print(sarimaxmod.summary())


## 3.1 Actual vs Fitted Values


In [ ]:
# Actual vs Fitted values — Real GDP Growth
fitted = sarimaxmod.fittedvalues

# Growth rate = difference of log GDP * 100
actual_growth = Y.diff() * 100
fitted_growth = fitted.diff() * 100

fig, ax = plt.subplots(figsize=(14, 5))
fig.patch.set_alpha(0); ax.patch.set_alpha(0); ax.grid(False)

ax.plot(actual_growth, color='blue', linewidth=1.0, label='Actual')
ax.plot(fitted_growth, color='red', linewidth=1.0, label='Fitted')
ax.axhline(0, color='grey', linewidth=0.4, linestyle='--')
ax.set_title('Actual vs Fitted — Real GDP Growth (SARIMAX)',
             fontsize=13, fontweight='bold')
ax.set_ylabel('Growth Rate (%)')
ax.set_xlabel('Date')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10),
          ncol=2, frameon=False)
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.show()


## 3.2 Model Diagnostics (4-Panel Plot)


In [ ]:
# Standard 4-panel diagnostics: standardized residuals, histogram,
# Q-Q plot, correlogram (Ljung-Box)
fig = sarimaxmod.plot_diagnostics(figsize=(14, 10))
fig.patch.set_alpha(0)
for ax in fig.axes:
    ax.patch.set_alpha(0)
    ax.grid(False)
    leg = ax.get_legend()
    if leg is not None:
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                  ncol=3, frameon=False)
plt.tight_layout()
plt.show()


## 4. Residual Diagnostics


In [ ]:
# ACF of SARIMAX residuals
fig, ax = plt.subplots(figsize=(12, 5))
fig.patch.set_alpha(0); ax.patch.set_alpha(0); ax.grid(False)
plot_acf(sarimaxmod.resid, lags=36, ax=ax, color='steelblue',
         vlines_kwargs={'colors': 'steelblue'})
ax.set_title('ACF of SARIMAX Residuals', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Ljung-Box test of SARIMAX residuals
fig, ax = plot_ljungbox_pvalues(sarimaxmod.resid, noestimatedcoef=1, nolags=36,
    title='Ljung-Box P-values — SARIMAX Residuals')
plt.show()
print('All p-values >> 0.05 → model is adequate ✅')


In [ ]:
# Ljung-Box test of squared residuals (ARCH effect)
fig, ax = plot_ljungbox_pvalues(sarimaxmod.resid**2, noestimatedcoef=0, nolags=36,
    title='Ljung-Box P-values — Squared SARIMAX Residuals (ARCH Check)')
plt.show()
print('No ARCH effect in the residual series ✅')


## Conclusion

The SARIMAX(1,0,0)(0,0,0)$_4$ model with `Lrealcons` and `Lrealinv` as exogenous regressors is parsimonious and adequate. Residuals resemble white noise (all Ljung-Box p-values > 0.05) and show no ARCH effects.
